In [ ]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=yymk81Bo0odf7J5MSnJ6rDrICbfCUjaQkfmVM7bc"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)



In [ ]:
import json
data = json.loads(response.text)
data

In [ ]:
import json
import pandas as pd
df_complexes = pd.json_normalize(data["complexes"],sep="_")
df_complexes

In [ ]:
df_complexes = df_complexes.rename(columns={"id": "complex_id", "name": "complex_name"})
df_complexes

In [ ]:
df_complexes = df_complexes[["complex_id", "complex_name" ]]
df_complexes

In [ ]:
# Extract Venues Data (handling cases where 'venues' key might be missing)
venues_data = [
    {
        "venue_id": v["id"],
        "venue_name": v["name"],
        "city_name": v["city_name"],
        "country_name": v["country_name"],
        "country_code": v["country_code"],
        "timezone": v["timezone"],
        "complex_id": c["id"]  # Foreign Key reference
    }
    for c in data["complexes"] if "venues" in c
    for v in c["venues"]
]
df_venues = pd.DataFrame(venues_data)

df_venues

In [ ]:
# Remove duplicates from Complexes DataFrame
df_complexes = df_complexes.drop_duplicates(subset=["complex_id"])

# Remove duplicates from Venues DataFrame
df_venues = df_venues.drop_duplicates(subset=["venue_id"])

In [ ]:
df_complexes

In [ ]:
df_venues

In [ ]:
import mysql.connector

# Establish connection to MySQL 
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="240996",
    database="Tennis_project"
)

cursor = conn.cursor()

create_complexes_table = """
CREATE TABLE IF NOT EXISTS Complexes (
    complex_id VARCHAR(50) PRIMARY KEY,
    complex_name VARCHAR(100) NOT NULL
);
"""
cursor.execute(create_complexes_table)

# Create Venues table with foreign key constraint
create_venues_table = """
CREATE TABLE IF NOT EXISTS Venues (
    venue_id VARCHAR(50) PRIMARY KEY,
    venue_name VARCHAR(100) NOT NULL,
    city_name VARCHAR(100) NOT NULL,
    country_name VARCHAR(100) NOT NULL,
    country_code CHAR(3) NOT NULL,
    timezone VARCHAR(100) NOT NULL,
    complex_id VARCHAR(50),
    FOREIGN KEY (complex_id) REFERENCES Complexes(complex_id) ON DELETE CASCADE
);
"""
cursor.execute(create_venues_table)

In [ ]:
complexes_insert_query = """
INSERT INTO Complexes (complex_id, complex_name) 
VALUES (%s, %s) 
ON DUPLICATE KEY UPDATE complex_name = VALUES(complex_name);
"""

complexes_data = list(df_complexes.itertuples(index=False, name=None))  # Convert DataFrame to list of tuples

cursor.executemany(complexes_insert_query, complexes_data)
conn.commit()

venues_insert_query = """
INSERT INTO Venues (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id) 
VALUES (%s, %s, %s, %s, %s, %s, %s) 
ON DUPLICATE KEY UPDATE 
    venue_name = VALUES(venue_name),
    city_name = VALUES(city_name),
    country_name = VALUES(country_name),
    country_code = VALUES(country_code),
    timezone = VALUES(timezone),
    complex_id = VALUES(complex_id);
"""

venues_data = list(df_venues.itertuples(index=False, name=None))  # Convert DataFrame to list of tuples

cursor.executemany(venues_insert_query, venues_data)
conn.commit()


In [ ]:
cursor.execute("select * from complexes")
for i in cursor:
    print(i)

In [ ]:
conn.commit()
cursor.close()
conn.close()